<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Low_Rank_Adaptation_(LoRA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class LoRALayer(nn.Module):
    def __init__(self, in_features, out_features, rank=4):
        super(LoRALayer, self).__init__()
        self.lora_A = nn.Parameter(torch.randn(in_features, rank))
        self.lora_B = nn.Parameter(torch.randn(rank, out_features))
        self.scale = 0.1

    def forward(self, x):
        return x + self.scale * (x @ self.lora_A @ self.lora_B)

class LoRAFoundationModel(nn.Module):
    def __init__(self, model):
        super(LoRAFoundationModel, self).__init__()
        self.model = model
        self.lora_layer = LoRALayer(in_features=768, out_features=768)

    def forward(self, input_ids, attention_mask=None):
        # Extract the last hidden state from the model output
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state  # Extracting the actual tensor data
        lora_output = self.lora_layer(hidden_state)
        return lora_output

# Example usage
# Load a pre-trained transformer model from Hugging Face
base_model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Initialize the LoRAFoundationModel with the base model
lora_model = LoRAFoundationModel(base_model)

# Create a dummy input
input_text = "This is an example sentence."
inputs = tokenizer(input_text, return_tensors="pt")

# Forward pass through the LoRAFoundationModel
outputs = lora_model(inputs["input_ids"], inputs["attention_mask"])
print(outputs)